## .tflite conversion

In [3]:
import tensorflow as tf
print(tf.VERSION)

1.12.0


In [4]:
import tensorflow as tf

def convert_hdf5_to_tflite(keras_file, filename):
    converter = tf.contrib.lite.TFLiteConverter.from_keras_model_file(keras_file)
    tflite_model = converter.convert()
    open(filename, "wb").write(tflite_model)

In [62]:
convert_hdf5_to_tflite('./keras_models/weights.best.from_scratch.hdf5', './tflite/model_from_scratch.tflite')

INFO:tensorflow:Froze 10 variables.
INFO:tensorflow:Converted 10 variables to const ops.


In [6]:
convert_hdf5_to_tflite('./keras_models/weights.extra_layers.hdf5', './tflite/extra_layers.tflite')

INFO:tensorflow:Froze 4 variables.
INFO:tensorflow:Converted 4 variables to const ops.


In [7]:
convert_hdf5_to_tflite('./keras_models/weights.best_single_model.hdf5', './tflite/best_single_model.tflite')

INFO:tensorflow:Froze 324 variables.
INFO:tensorflow:Converted 324 variables to const ops.


In [15]:
input_shapes = { 'input_3': [None, 224, 244, 3] }
convert_hdf5_to_tflite('./keras_models/base_resnet.hdf5', './tflite/base_resnet.tflite', input_shapes)

{'input_3': [None, 224, 244, 3]}
INFO:tensorflow:Froze 318 variables.
INFO:tensorflow:Converted 318 variables to const ops.


ValueError: None is only supported in the 1st dimension. Tensor 'input_3' has invalid shape '[None, None, None, 3]'.

In [26]:
from keras.models import load_model

base_resnet = load_model('./keras_models/base_resnet.hdf5')
base_resnet.inputs[0]

/home/akshata/.local/lib/python3.6/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


<tf.Tensor 'input_3_4:0' shape=(?, ?, ?, 3) dtype=float32>

In [38]:
## update base_resnet input shape to be compatible with tflite

from keras.models import Input, Model

def change_input_shape(model, input_shape = (None, 224, 224, 3)):
    model.layers.pop(0)
    newInput = Input(batch_shape=input_shape)    
    newOutputs = base_resnet(newInput)
    newModel = Model(newInput, newOutputs)
    newModel.summary()
    return newModel

In [58]:
## create a new model
from keras.models import Sequential, InputLayer

# Start construction of the Keras Sequential model.
test_model = Sequential()

# Add an input layer which is similar to a feed_dict in TensorFlow.
# Note that the input-shape must be a tuple containing the image-size.
# test_model.add(InputLayer(input_shape=(224, 244, 3)))

for layer in base_resnet.layers:
    layer.trainable = False
    test_model.add(layer)

test_model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
test_model.fit([], [])  # starts training
test_model.summary()

AssertionError: 

## .pb Conversion

Deploying to android requires a .pb format. Use the following function, to convert your model 

### Method 1

In [ ]:
## not compatible with old versions of tensorflow
import os
from tensorflow.python.framework import graph_util, graph_io
from tensorflow.python.tools import import_pb_to_tensorboard

def keras_to_tensorflow(keras_model, 
        output_dir, 
        model_name,
        out_prefix="output_", 
        log_tensorboard=True):

    if os.path.exists(output_dir) == False:
        os.mkdir(output_dir)

    out_nodes = []

    for i in range(len(keras_model.outputs)):
        out_nodes.append(out_prefix + str(i + 1))
        tf.identity(keras_model.output[i], 
            out_prefix + str(i + 1))

    sess = K.get_session()


    init_graph = sess.graph.as_graph_def()

    main_graph = graph_util.convert_variables_to_constants(
            sess, init_graph, out_nodes)

    graph_io.write_graph(main_graph, output_dir, 
        name=model_name, as_text=False)

    if log_tensorboard:
        import_pb_to_tensorboard.import_to_tensorboard(
            os.path.join(output_dir, model_name),
            output_dir)

### Method 2: Freeze Session

In [9]:
## not compatible with old versions of tensorflow
import os
from tensorflow.python.framework import graph_util, graph_io
from tensorflow.python.tools import import_pb_to_tensorboard

def freeze_session(keras_model, 
        output_dir, 
        model_name,
        out_prefix="output_", 
        log_tensorboard=True):

    # todo
    pass

In [ ]:
def save_frozen_graph(model, name, filename):
    frozen_graph = freeze_session(K.get_session(),
                              output_names=[out.op.name for out in model.outputs])
    tf.train.write_graph(frozen_graph, name, filename, as_text=False)

In [ ]:
## save both models
import os
import tensorflow as tf
from keras import backend as K

save_frozen_graph(base_resnet, 'android', 'base_resnet_model.pb')
save_frozen_graph(last_layers_model, 'last_layers', 'last_layers_model.pb')

# keras_to_tensorflow(base_resnet, 'android', 'base_resnet_model.pb')
# keras_to_tensorflow(last_layers_model, 'android', 'last_layers_model.pb')

### Method 3 (from Siraj Raval Video)

In [ ]:
from tensorflow.python.tools import freeze_graph
from tensorflow.python.tools import optimize_for_inference_lib

def export_model(saver, model, input_node_names, output_node_name, model_name):
    tf.train.write_graph(K.get_session().graph_def, 'out', \
        model_name + '_graph.pbtxt')

    saver.save(K.get_session(), 'out/' + model_name + '.chkp')

    freeze_graph.freeze_graph('out/' + model_name + '_graph.pbtxt', None, \
        False, 'out/' + model_name + '.chkp', output_node_name, \
        "save/restore_all", "save/Const:0", \
        'out/frozen_' + model_name + '.pb', True, "")

    input_graph_def = tf.GraphDef()
    with tf.gfile.Open('out/frozen_' + model_name + '.pb', "rb") as f:
        input_graph_def.ParseFromString(f.read())

    output_graph_def = optimize_for_inference_lib.optimize_for_inference(
            input_graph_def, input_node_names, [output_node_name],
            tf.float32.as_datatype_enum)

    with tf.gfile.FastGFile('out/opt_' + model_name + '.pb', "wb") as f:
        f.write(output_graph_def.SerializeToString())

    print("graph saved!")
    
export_model(tf.train.Saver(), last_layers_model, ["flatten_7_input"], 'dense_16/Softmax', 'last_layers')

## Getting input / output names of model

In [ ]:
[node.op.name for node in last_layers_model.inputs]

## Testing if the probuf models were saved corrected

In [ ]:
import tensorflow as tf 

def test_protobuf(filepath):
    g = tf.GraphDef()
    g.ParseFromString(open(filepath, 'rb').read())

    # same for output or any other node you want to make sure is ok
    print()
    print(filepath)
    print("=======================INPUT===================")
    print([n for n in g.node if n.name.find('input') != -1])
    print("=======================OUTPUT==================")
    print([n for n in g.node if n.name.find('output') != -1])
    print("===================KERAS_LEARNING==============")
    print([n for n in g.node if n.name.find('keras_learning_phase') != -1])
    print("===============================================")
    print()

In [ ]:
resnet_pb = 'android/base_resnet_model.pb'
ex_layers_pb = 'android/last_layers_model.pb'

test_protobuf(resnet_pb)
test_protobuf(ex_layers_pb)